<a href="https://colab.research.google.com/github/Natural-Language-Processing-YU/M3_Assignment/blob/main/scripts/m3_assignment_part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [ ]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
None
### END CODE HERE ###

In [ ]:
!pip install tensorflow_datasets
!pip install -U tensorflow-text

In [ ]:
pip install tensorflow

# **Encoder-Decoder Approach (Transformer-based BERT model)**

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

# Load data
male_names = ['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis', 'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel', 'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil']
female_names = ['Alice', 'Alicia', 'Alina', 'Alison', 'Alissa', 'Allyson', 'Alma', 'Althea', 'Alva', 'Alyson', 'Alyssa', 'Amber', 'Amelia', 'Amie', 'Amy', 'Ana', 'Anastasia', 'Andrea', 'Angel', 'Angela', 'Angelia', 'Angelica', 'Angelina', 'Angeline']
all_names = male_names + female_names


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

name_encodings = tokenizer(all_names, truncation=True, padding=True, max_length=10, return_tensors='pt')


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(20):  # You can increase the number of epochs for better performance
    optimizer.zero_grad()
    train_outputs = model(**name_encodings, labels=torch.tensor([0]*len(male_names) + [1]*len(female_names)))
    train_loss = train_outputs.loss
    train_loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss.item()}')

# Get predicted labels
model.eval()
outputs = model(**name_encodings)
_, predicted = torch.max(outputs.logits, 1)

# Print predicted names
print("Predicted Names:")
for name, prediction in zip(all_names, predicted):
    predicted_gender = "male" if prediction.item() == 0 else "female"
    print(f'{name}: {predicted_gender}')



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.7257802486419678
Epoch 2, Train Loss: 0.6988059282302856
Epoch 3, Train Loss: 0.6930409669876099
Predicted Names:
Abbey: male
Abbie: male
Abby: male
Addie: male
Adrian: male
Adrien: male
Ajay: male
Alex: male
Alexis: male
Alfie: male
Ali: male
Alix: male
Allie: male
Allyn: male
Andie: male
Andrea: male
Andy: male
Angel: male
Angie: male
Ariel: male
Ashley: male
Aubrey: male
Augustine: male
Austin: male
Averil: male
Alice: male
Alicia: male
Alina: female
Alison: male
Alissa: female
Allyson: female
Alma: male
Althea: female
Alva: female
Alyson: female
Alyssa: male
Amber: male
Amelia: male
Amie: female
Amy: male
Ana: female
Anastasia: female
Andrea: male
Angel: male
Angela: male
Angelia: female
Angelica: male
Angelina: male
Angeline: female


# **LSTM-based Approach**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Load data
male_names = ['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis', 'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel', 'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil']
female_names = ['Alice', 'Alicia', 'Alina', 'Alison', 'Alissa', 'Allyson', 'Alma', 'Althea', 'Alva', 'Alyson', 'Alyssa', 'Amber', 'Amelia', 'Amie', 'Amy', 'Ana', 'Anastasia', 'Andrea', 'Angel', 'Angela', 'Angelia', 'Angelica', 'Angelina', 'Angeline']
all_names = male_names + female_names
labels = [0] * len(male_names) + [1] * len(female_names)

# Encode names
char_to_idx = {char: idx for idx, char in enumerate('abcdefghijklmnopqrstuvwxyz')}
max_len = max(len(name) for name in all_names)
def encode_name(name):
    name = name.lower()
    encoded = torch.zeros(max_len, 26)
    for i, char in enumerate(name):
        encoded[i][char_to_idx[char]] = 1
    return encoded

encoded_names = [encode_name(name) for name in all_names]

# Split data
X_train, X_test, y_train, y_test = train_test_split(encoded_names, labels, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = torch.stack(X_train)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.stack(X_test)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_size=26, hidden_size=128, num_layers=2, num_classes=2).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct_preds / total_preds

    # Validation
    model.eval()
    val_running_loss = 0.0
    val_correct_preds = 0
    val_total_preds = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            val_loss = criterion(outputs, labels)

            val_running_loss += val_loss.item()

            _, predicted = torch.max(outputs, 1)
            val_correct_preds += (predicted == labels).sum().item()
            val_total_preds += labels.size(0)

    val_loss = val_running_loss / len(val_loader)
    val_acc = val_correct_preds / val_total_preds

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Test the model and print test accuracy and classification results
model.eval()
test_correct_preds = 0
test_total_preds = 0
predicted_classes = []
with torch.no_grad():  # Disable gradient calculation during inference
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct_preds += (predicted == labels).sum().item()
        test_total_preds += labels.size(0)

        # Map predicted labels to class names and store them
        predicted_classes.extend([class_names[label.item()] for label in predicted])

test_acc = test_correct_preds / test_total_preds
print(f'Test Accuracy: {test_acc:.4f}')

# Print lengths for debugging
print(f'Length of all_names: {len(all_names)}')
print(f'Length of predicted_classes: {len(predicted_classes)}')

# Print each name along with its predicted class
print("Predictions:")
for i, (name, pred_class) in enumerate(zip(all_names, predicted_classes)):
    print(f'{name}: {pred_class}')


Epoch 1/10, Train Loss: 0.6973, Train Acc: 0.4103, Val Loss: 0.7009, Val Acc: 0.3000
Epoch 2/10, Train Loss: 0.6876, Train Acc: 0.5641, Val Loss: 0.7113, Val Acc: 0.3000
Epoch 3/10, Train Loss: 0.6828, Train Acc: 0.5641, Val Loss: 0.7257, Val Acc: 0.3000
Epoch 4/10, Train Loss: 0.6813, Train Acc: 0.5641, Val Loss: 0.7498, Val Acc: 0.3000
Epoch 5/10, Train Loss: 0.6811, Train Acc: 0.5641, Val Loss: 0.7598, Val Acc: 0.3000
Epoch 6/10, Train Loss: 0.6683, Train Acc: 0.5641, Val Loss: 0.7383, Val Acc: 0.3000
Epoch 7/10, Train Loss: 0.6566, Train Acc: 0.5641, Val Loss: 0.7167, Val Acc: 0.3000
Epoch 8/10, Train Loss: 0.6376, Train Acc: 0.5641, Val Loss: 0.6971, Val Acc: 0.3000
Epoch 9/10, Train Loss: 0.5844, Train Acc: 0.5897, Val Loss: 0.6229, Val Acc: 0.4000
Epoch 10/10, Train Loss: 0.4889, Train Acc: 0.7949, Val Loss: 0.5219, Val Acc: 0.7000
Test Accuracy: 0.7000
Length of all_names: 49
Length of predicted_classes: 10
Predictions:
Abbey: female
Abbie: female
Abby: female
Addie: female
Adr

# References
1. https://arxiv.org/pdf/2102.03692.pdf
2. https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/exercise/13-attention.html
3. https://towardsdatascience.com/deep-learning-gender-from-name-lstm-recurrent-neural-networks-448d64553044
4. https://www.nltk.org/book/ch02.html#sec-lexical-resources